In [2]:
pip install --upgrade gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 24.1 MB 61.5 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import pandas as pd
import numpy as np
import ast
import re
import sklearn
import six
from abc import ABCMeta
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from scipy import sparse
import warnings
warnings.filterwarnings('ignore')
import nltk
import six
import re
import gensim
from tqdm import tqdm


from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.svm import LinearSVC
from abc import ABCMeta
from sklearn.pipeline import Pipeline
from sklearn.utils import check_X_y, check_array
from scipy import sparse
from nltk.stem import PorterStemmer,WordNetLemmatizer
from scipy.sparse import issparse
from sklearn.base import BaseEstimator, ClassifierMixin
from sklearn.utils.extmath import safe_sparse_dot
from sklearn.preprocessing import normalize, binarize, LabelBinarizer
from sklearn.feature_extraction.text import CountVectorizer,TfidfVectorizer,TfidfTransformer
from sklearn.utils import resample
from sklearn.metrics import accuracy_score,f1_score
from gensim.models import FastText

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
tsel = pd.read_csv(r'/content/drive/MyDrive/TA/Prepro_Pelayanan.csv', usecols=['Username','Text','Pelayanan','text_result'])
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,"['handphone', 'lapor', 'wajib', 'bantu', 'laca..."
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,"['tidak tahu', 'telkomsel', 'lambat', 'kuota',..."
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,"['kendala', 'sinyal', 'telkomsel']"
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,"['tidak bisa', 'nomor', 'mati', 'dihidupakan',..."
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,"['admin', 'kemarin', 'aplikasi', 'punya', 'tel..."
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,"['sinyal', 'minus', 'tolong', 'baik', 'problem..."
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,"['punya', 'telkomsel', 'ganggu', 'aduh', 'desa..."
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,"['allah', 'tambah', 'kerja', 'esuk', 'neng', '..."
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,"['admin', 'sinyal', 'telkomsel', 'rumah', 'dae..."


In [6]:
tsel['text_result'] = tsel['text_result'].apply(str)

In [7]:
import ast

def join_text_list(text):
    texts = ast.literal_eval(text)
    return ' '.join([text for text in texts])

tsel['text_result'] = tsel['text_result'].apply(join_text_list)
tsel

,Username,Text,Pelayanan,text_result
0,satriafakhri_id,@yamasaiyaa untuk masalah hp lapor aja ke yg b...,0.0,handphone lapor wajib bantu lacak imei perihal...
1,Piyuuuuu21_,"@bertanyarl Gatau, akhir akhir ini Telkomsel l...",0.0,tidak tahu telkomsel lambat kuota promo murah ...
2,hypersex_pr,Ada kendala kah dengan sinyal telkomsel,0.0,kendala sinyal telkomsel
3,snwyscrtimh,@squidwardpemala @Cutimut3 @reihanahsan @sonde...,-1.0,tidak bisa nomor mati dihidupakan grapari nomo...
4,aninadhany,"@Telkomsel min, dari kmrn aplikasi my telkomse...",-1.0,admin kemarin aplikasi punya telkomsel ganggu ...
...,...,...,...,...
16986,Syaeeef,"Sinyal mines, #sinyal #telkomsel #byu.id tolon...",0.0,sinyal minus tolong baik problem location cama...
16987,abdr40,@Telkomsel ini My Telkomsel gangguan sampe kap...,0.0,punya telkomsel ganggu aduh desak pakai
16988,thepinediary,yaallah nambah2i gawean waeee esuk2 kudu neng ...,0.0,allah tambah kerja esuk neng grapari
16989,yudaamaraa,@Telkomsel min sinyal telkomsel di rumah saya ...,0.0,admin sinyal telkomsel rumah daerah rawabunga ...


## Ektraksi Fitur TF-IDF 

In [8]:
vectors= TfidfVectorizer(ngram_range=(1,2), max_features = 1000)
vectors

TfidfVectorizer(max_features=1000, ngram_range=(1, 2))

In [9]:
tfidf = vectors.fit_transform(tsel['text_result'].astype(str))
tfidf

<16991x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 116944 stored elements in Compressed Sparse Row format>

In [10]:
df_tfidf = tfidf.copy()

In [11]:
X = df_tfidf
y = tsel['Pelayanan']

In [12]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(X,y, test_size=0.1, random_state=5)

## Ekspansi Fitur

In [ ]:
# Load Model trained
model_exp = FastText.load('/content/drive/MyDrive/TA/model_ntwt/fasttext/Twitter_Berita.fasttext').wv 
# penggunaan corpus disesuaikan (corpus Twitter, Berita, Twitter+bero)

In [ ]:
def feature_expansion(df,feature):
    for col in tqdm(df.columns): #loop per kolom
        try: 
            sim_word = model_exp.similar_by_word(col, topn = 20) #Mencari similarity berdasarkan nilai n
        except:
            sim_word = []
        if sim_word != []: #jika similarity tidak kosong
            for term in [sim_word[i][0] for i in range(len(sim_word))]: #loop per-word yang ada di Similarity
                if term in feature:
                    #untuk semua kolom yang mempunyai nilai 0 di kolom, tetapi mempunyai nilai yang bukan 0 pada kolom term
                    #nilainya diganti dengan nilai kolom term yang mempunyai nilai bukan 0
                    df[col][(df[col]==0) & (df[term]!=0)] = df[term][(df[col]==0) & (df[term]!=0)]
    return df

In [ ]:
feature_ef = vectors.get_feature_names()

In [ ]:
# Membuat dataframe data test & train
tsel_x_train = pd.DataFrame(x_train.todense(), columns = feature_ef)
tsel_x_test = pd.DataFrame(x_test.todense(), columns = feature_ef)

In [ ]:
dfexp_x_train= feature_expansion(tsel_x_train, feature_ef)

100%|██████████| 1000/1000 [00:12<00:00, 77.41it/s]


In [ ]:
dfexp_x_test = feature_expansion(tsel_x_test, feature_ef)

100%|██████████| 1000/1000 [00:11<00:00, 88.88it/s]


## Klasifikasi

In [ ]:
class NBSVM(six.with_metaclass(ABCMeta, BaseEstimator, ClassifierMixin)):
    def __init__(self, alpha=1.0, C=1.0, max_iter=10000):
        self.alpha = alpha
        self.max_iter = max_iter
        self.C = C
        self.svm_ = [] # fuggly

    def fit(self, X, y):
        X, y = check_X_y(X, y, 'csr')
        _, n_features = X.shape

        labelbin = LabelBinarizer()
        Y = labelbin.fit_transform(y)
        self.classes_ = labelbin.classes_
        if Y.shape[1] == 1:
            Y = np.concatenate((1 - Y, Y), axis=1)

        # LabelBinarizer().fit_transform() returns arrays with dtype=np.int64.
        # so we don't have to cast X to floating point
        Y = Y.astype(np.float64)

        # Count raw events from data
        n_effective_classes = Y.shape[1]
        self.class_count_ = np.zeros(n_effective_classes, dtype=np.float64)
        self.ratios_ = np.full((n_effective_classes, n_features), self.alpha,
                                 dtype=np.float64)
        self._compute_ratios(X, Y)

        # flugglyness
        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            svm = LinearSVC(C=self.C, max_iter=self.max_iter)
            Y_i = Y[:,i]
            svm.fit(X_i, Y_i)
            self.svm_.append(svm) 

        return self

    def predict(self, X):
        n_effective_classes = self.class_count_.shape[0]
        n_examples = X.shape[0]

        D = np.zeros((n_effective_classes, n_examples))

        for i in range(n_effective_classes):
            X_i = X.multiply(self.ratios_[i])
            D[i] = self.svm_[i].decision_function(X_i)
        
        return self.classes_[np.argmax(D, axis=0)]
        
    def _compute_ratios(self, X, Y):
        """Count feature occurrences and compute ratios."""
        if np.any((X.data if issparse(X) else X) < 0):
            raise ValueError("Input X must be non-negative")

        self.ratios_ += safe_sparse_dot(Y.T, X)  # ratio + feature_occurrance_c
        normalize(self.ratios_, norm='l1', axis=1, copy=False)
        row_calc = lambda r: np.log(np.divide(r, (1 - r)))
        self.ratios_ = np.apply_along_axis(row_calc, axis=1, arr=self.ratios_)
        check_array(self.ratios_)
        self.ratios_ = sparse.csr_matrix(self.ratios_)

In [ ]:
x_train_csr = sparse.csr_matrix(dfexp_x_train)
x_test_csr = sparse.csr_matrix(dfexp_x_test)
model = NBSVM()

model.fit(x_train_csr,y_train)
y_pred_exp = model.predict(x_test_csr)
y_true = y_test

### Top 1 

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true,y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true,y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[ 143   91   11]
 [  49 1242   20]
 [  16   76   52]]
              precision    recall  f1-score   support

        -1.0       0.69      0.58      0.63       245
         0.0       0.88      0.95      0.91      1311
         1.0       0.63      0.36      0.46       144

    accuracy                           0.85      1700
   macro avg       0.73      0.63      0.67      1700
weighted avg       0.83      0.85      0.83      1700

Precision:  0.7318274161229731
Recall:  0.6307176671838326
F1:  0.6675772174065919
Accuracy:  0.8452941176470589


### Top 5

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[ 141   94   10]
 [  54 1233   24]
 [  13   76   55]]
              precision    recall  f1-score   support

        -1.0       0.68      0.58      0.62       245
         0.0       0.88      0.94      0.91      1311
         1.0       0.62      0.38      0.47       144

    accuracy                           0.84      1700
   macro avg       0.72      0.63      0.67      1700
weighted avg       0.83      0.84      0.83      1700

Precision:  0.7248977399132164
Recall:  0.6326526936734521
F1:  0.6677471735965247
Accuracy:  0.8405882352941176


### Top 10

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[ 140   89   16]
 [  47 1245   19]
 [  16   76   52]]
              precision    recall  f1-score   support

        -1.0       0.69      0.57      0.62       245
         0.0       0.88      0.95      0.92      1311
         1.0       0.60      0.36      0.45       144

    accuracy                           0.85      1700
   macro avg       0.72      0.63      0.66      1700
weighted avg       0.83      0.85      0.83      1700

Precision:  0.7234450150811119
Recall:  0.627398811037255
F1:  0.6634403970401765
Accuracy:  0.8452941176470589


### Top 20

In [ ]:
#example training
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
#[None, 'micro', 'macro', 'weighted'].
print(confusion_matrix(y_true,y_pred_exp))
print(classification_report(y_true,y_pred_exp))
print("Precision: ",precision_score(y_true, y_pred_exp, average="macro"))
print("Recall: ",recall_score(y_true, y_pred_exp, average="macro"))
print("F1: ",f1_score(y_true, y_pred_exp,  average="macro"))
print("Accuracy: ",accuracy_score(y_true, y_pred_exp))

[[ 140   96    9]
 [  46 1244   21]
 [  19   73   52]]
              precision    recall  f1-score   support

        -1.0       0.68      0.57      0.62       245
         0.0       0.88      0.95      0.91      1311
         1.0       0.63      0.36      0.46       144

    accuracy                           0.84      1700
   macro avg       0.73      0.63      0.67      1700
weighted avg       0.83      0.84      0.83      1700

Precision:  0.7324898302061578
Recall:  0.6271445522017605
F1:  0.6652539717605407
Accuracy:  0.8447058823529412
